In [1]:
import argparse
import logging
import sys
import warnings
from datetime import datetime
from pathlib import Path
from typing import List, Tuple
from itertools import chain

import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

from utils.dataloaders import (
    load_clarity_data,
    load_aladdin_data,
    load_crossreference,
    load_portfolios,
    load_overrides,
)

from utils.get_date import get_date
from utils.set_up_log import set_up_log

In [2]:
# Set up logging
logger = set_up_log("Pre-OVR-Analysis")
# Ignore workbook warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [15]:
def prepare_dataframes(
    df1: pd.DataFrame, df2: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Prepare DataFrames by setting index and filtering for common indexes.
    Logs info about common, new, and missing indexes.
    """
    # Set index to 'permid' if it exists, otherwise assume it's already the index.
    if "permid" in df1.columns:
        df1 = df1.set_index("permid")
    else:
        logging.warning("df1 does not contain a 'permid' column. Using current index.")
        
    if "permid" in df2.columns:
        df2 = df2.set_index("permid")
    else:
        logging.warning("df2 does not contain a 'permid' column. Using current index.")
    
    common_indexes = df1.index.intersection(df2.index)
    new_indexes = df2.index.difference(df1.index)
    missing_indexes = df1.index.difference(df2.index)
    
    logging.info(f"Number of common indexes: {len(common_indexes)}")
    
    return (
        df1.loc[common_indexes],
        df2.loc[common_indexes],
        df2.loc[new_indexes],
        df1.loc[missing_indexes],
    )


In [4]:
def compare_dataframes(
    df1: pd.DataFrame, df2: pd.DataFrame, test_col: List[str]
) -> pd.DataFrame:
    """Compare DataFrames and create delta DataFrame."""
    delta = df2.copy()
    for col in test_col:
        if col in df1.columns and col in df2.columns:
            logging.info(f"Comparing column: {col}")
            diff_mask = df1[col] != df2[col]
            delta.loc[~diff_mask, col] = np.nan
    return delta


def get_exclusion_list(
    row: pd.Series, df1: pd.DataFrame, test_col: List[str]
) -> List[str]:
    """Get list of columns that changed to EXCLUDED."""
    return [
        col
        for col in test_col
        if row[col] == "EXCLUDED" and df1.loc[row.name, col] != "EXCLUDED"
    ]


def get_inclusion_list(
    row: pd.Series, df1: pd.DataFrame, test_col: List[str]
) -> List[str]:
    """Get list of columns that changed from EXCLUDED to any other value."""
    return [
        col
        for col in test_col
        if row[col] != "EXCLUDED" and df1.loc[row.name, col] == "EXCLUDED"
    ]


def check_new_exclusions(
    df1: pd.DataFrame, df2: pd.DataFrame, delta: pd.DataFrame, test_col: List[str]
) -> pd.DataFrame:
    """Check for new exclusions and update delta DataFrame."""
    delta["new_exclusion"] = False
    for col in test_col:
        if col in df1.columns and col in df2.columns:
            logging.info(f"Checking for new exclusions in column: {col}")
            mask = (df1[col] != "EXCLUDED") & (df2[col] == "EXCLUDED")
            delta.loc[mask, "new_exclusion"] = True
            logging.info(f"Number of new exclusions in {col}: {mask.sum()}")
    delta["exclusion_list"] = delta.apply(
        lambda row: get_exclusion_list(row, df1, test_col), axis=1
    )
    return delta


def check_new_inclusions(
    df1: pd.DataFrame, df2: pd.DataFrame, delta: pd.DataFrame, test_col: List[str]
) -> pd.DataFrame:
    """Check for new inclusions and update delta DataFrame."""
    delta["new_inclusion"] = False
    for col in test_col:
        if col in df1.columns and col in df2.columns:
            logging.info(f"Checking for new inclusions in column: {col}")
            mask = (df1[col] == "EXCLUDED") & (df2[col] != "EXCLUDED")
            delta.loc[mask, "new_inclusion"] = True
            logging.info(f"Number of new inclusions in {col}: {mask.sum()}")
    delta["inclusion_list"] = delta.apply(
        lambda row: get_inclusion_list(row, df1, test_col), axis=1
    )
    return delta


def finalize_delta(delta: pd.DataFrame, test_col: List[str]) -> pd.DataFrame:
    """Finalize delta DataFrame by removing unchanged rows and resetting index."""
    delta = delta.dropna(subset=test_col, how="all")
    delta.reset_index(inplace=True)
    logging.info(f"Final delta shape: {delta.shape}")
    return delta


In [5]:
# Get user input for date
#DATE = get_date()
#YEAR = DATE[:4]
#date_obj = datetime.strptime(DATE, "%Y%m")
#prev_date_obj = date_obj - relativedelta(months=1)
#DATE_PREV = prev_date_obj.strftime("%Y%m")


In [6]:
# TEMP DATE CONST
DATE = "202503"
YEAR = DATE[:4]
DATE_PREV = "202502"

# DEFINE TEST COLUMNS
test_col = [
    "str_001_s",
    "str_002_ec",
    "str_003_ec",
    "str_004_asec",
    "str_005_ec",
    "cs_001_sec",
    "gp_esccp",
    "cs_003_sec",
    "cs_002_ec",
    "str_006_sec",
    "str_007_sect",
    "gp_esccp_22",
    "gp_esccp_25",
    "gp_esccp_30",
    "art_8_basicos",
    "str_003b_ec",
]

In [7]:
# DEFINE PATHS
REPO_DIR = Path(r"C:\Users\n740789\Documents\clarity_data_quality_controls")
DATAFEED_DIR = Path(r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED")
df_1_path = (
    DATAFEED_DIR
    / "datafeeds_with_ovr"
    / f"{DATE_PREV}_df_issuer_level_with_ovr.csv"
)
df_2_path = (
    DATAFEED_DIR
    / "ficheros_tratados"
    / f"{YEAR}"
    / f"{DATE}01_Equities_feed_IssuerLevel_sinOVR.csv"
)
ALADDIN_DATA_DIR = REPO_DIR / "excel_books" / "aladdin_data"
CROSSREFERENCE_PATH = (
    ALADDIN_DATA_DIR
    / "crossreference"
    / f"Aladdin_Clarity_Issuers_{DATE}01.csv"
)
BMK_PORTF_STR_PATH = (
    ALADDIN_DATA_DIR
    / "bmk_portf_str"
    / f"{DATE}_strategies_snt world_portf_bmks.xlsx"
)
SRI_DATA_DIR = REPO_DIR / "excel_books" / "sri_data"
OVR_PATH = (
    REPO_DIR
    / "excel_books"
    / "sri_data"
    / "overrides"
    / "20250318_overrides_db.xlsx"
)
COMMITTEE_PATH = (
    REPO_DIR
    / "excel_books"
    / "sri_data"
    / "portfolios_committees"
    / "portfolio_lists.xlsx"
)

In [8]:
# LOAD DATA
columns_to_read = ["permid", "isin", "issuer_name"] + test_col
# clarity data
df_1 = load_clarity_data(df_1_path, columns_to_read)
df_2 = load_clarity_data(df_2_path, columns_to_read)

In [9]:
# aladdin /brs data / perimetros
brs_carteras = load_aladdin_data(BMK_PORTF_STR_PATH, "portfolio_carteras")    
brs_benchmarks = load_aladdin_data(BMK_PORTF_STR_PATH, "portfolio_benchmarks")
crosreference = load_crossreference(CROSSREFERENCE_PATH)

In [23]:
crosreference.head()

,aladdin_id,issuer_name,permid,msci,sust
0,H56976,AUXIFIP SA,5001248970,IID000000002682941,NaN
1,H57042,AVESTA TECHNOLOGIES LLC,4295900331,NaN,NaN
2,H57890,INFANT BACTERIAL THERAPEUTICS AB,5040202605,IID000000002761045,2.004151e+09
3,H57901,MB SECURITIES JSC,4298118784,IID000000002761038,NaN
4,H57917,BARCODE 121 HOLDING AS,5050698850,NaN,NaN


In [10]:
# sri/ESG Team data
overrides = load_overrides(OVR_PATH)

loading overrides columns ['clarityid', 'permid', 'brs_id', 'ovr_target', 'ovr_value']


In [11]:
# Load portfolios & benchmarks dicts and lists
(
    portfolios_dict,
    benchmarks_dict,
    carteras_list,
    benchmarks_list,
    carteras_benchmarks_list,
) = load_portfolios(BMK_PORTF_STR_PATH)
logging.info(f"df_1 shape: {df_1.shape}, df_2 shape: {df_2.shape}")

2025-03-20 16:33:46,058 - root - INFO - df_1 shape: (69264, 19), df_2 shape: (69278, 19)


In [16]:
# PREPARE DATA
(
    df_1, 
    df_2,
    new_issuer,
    out_issuer,
) = prepare_dataframes(df_1, df_2)

# log size of new and missing issuers
logging.info(f"Number of new issuers: {new_issuer.shape[0]}")
logging.info(f"Number of missing issuers: {out_issuer.shape[0]}")

2025-03-20 16:54:06,510 - root - WARNING - df1 does not contain a 'permid' column. Using current index.
2025-03-20 16:54:06,512 - root - WARNING - df2 does not contain a 'permid' column. Using current index.
2025-03-20 16:54:06,550 - root - INFO - Number of common indexes: 69213
2025-03-20 16:54:06,626 - root - INFO - Number of new issuers: 65
2025-03-20 16:54:06,627 - root - INFO - Number of missing issuers: 51


In [31]:
out_issuer.head()


,isin,issuer_name,str_001_s,str_002_ec,str_003_ec,str_004_asec,str_005_ec,cs_001_sec,gp_esccp,cs_003_sec,cs_002_ec,str_006_sec,str_007_sect,gp_esccp_22,gp_esccp_25,gp_esccp_30,art_8_basicos,str_003b_ec
permid,,,,,,,,,,,,,,,,,,
4295857675,AU0000037889,Excelsior Capital Ltd,OK,OK,OK,EXCLUDED,OK,EXCLUDED,OK,EXCLUDED,OK,EXCLUDED,EXCLUDED,OK,OK,OK,OK,OK
4295857792,NO0004253238,Grand Hotel Management Pty Ltd,OK,OK,OK,EXCLUDED,OK,EXCLUDED,OK,EXCLUDED,OK,OK,EXCLUDED,OK,OK,OK,OK,OK
4295875321,IT0001082665,Comau SpA,OK,OK,OK,OK,OK,OK,OK,EXCLUDED,OK,OK,OK,OK,OK,OK,OK,OK
4295878072,JP3762100000,Novarese Inc (Pre-Merger),OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
4295884500,MA0001006560,Societe Equipement Domestique et Menager SA,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK


In [34]:
for i in sorted(out_issuer.issuer_name.unique()):
    print(i)

AceBiomed Inc
Adani Properties Pvt Ltd
American Resources Offshore  Inc
American Savings Bank FSB
Arch Western Finance LLC
Autostock Inc
Bank Of Madera County
Belron UK Finance PLC
COMAU SCIAKY SA
Central Valley Community Bank
China Huarong Financial Leasing Co Ltd
Comau SpA
Credit Suisse Funds AG
Credit Suisse Private Advisors AG in Liquidation
DG Liquidation Corp
Emerald Haven Town and Country Pvt Ltd
Excelsior Capital Ltd
Express LLC
Folsom Lake Bank
Fundacion Bancaria Caja de Ahorros de Asturias
Grab A Grub Services Pvt Ltd
Grand Hotel Management Pty Ltd
Hertz Fleet Lease Funding LP Series 2016 1
Hitit Bilgisayar Hizmetleri AS
Home Trust Co
Hunter Ridge Holdings Inc
Ignite Epm Live Solutions Inc
Integracao Transmissora de Energia SA
International Energy Group LLC
Jiminy Inc
La Marocaine Vie
Marocaine De Banques Ste Generale SA
Mercado Credito Sociedade de Credito Financiamento e Investimento SA
Novarese Inc (Pre-Merger)
Orbita Funding 2020-1 PLC
Pocheon Power Co Ltd
RHB Securities 

In [18]:
# COMPARE DATA
delta = compare_dataframes(df_1, df_2, test_col)
delta = check_new_exclusions(df_1, df_2, delta, test_col)
delta = check_new_inclusions(df_1, df_2, delta, test_col)
delta = finalize_delta(delta, test_col)


2025-03-20 16:57:43,795 - root - INFO - Comparing column: str_001_s
2025-03-20 16:57:43,815 - root - INFO - Comparing column: str_002_ec
2025-03-20 16:57:43,830 - root - INFO - Comparing column: str_003_ec
2025-03-20 16:57:43,847 - root - INFO - Comparing column: str_004_asec
2025-03-20 16:57:43,863 - root - INFO - Comparing column: str_005_ec
2025-03-20 16:57:43,879 - root - INFO - Comparing column: cs_001_sec
2025-03-20 16:57:43,895 - root - INFO - Comparing column: gp_esccp
2025-03-20 16:57:43,912 - root - INFO - Comparing column: cs_003_sec
2025-03-20 16:57:43,926 - root - INFO - Comparing column: cs_002_ec
2025-03-20 16:57:43,941 - root - INFO - Comparing column: str_006_sec
2025-03-20 16:57:43,958 - root - INFO - Comparing column: str_007_sect
2025-03-20 16:57:43,972 - root - INFO - Comparing column: gp_esccp_22
2025-03-20 16:57:43,985 - root - INFO - Comparing column: gp_esccp_25
2025-03-20 16:57:44,002 - root - INFO - Comparing column: gp_esccp_30
2025-03-20 16:57:44,014 - root

In [30]:
overrides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7671 entries, 0 to 7670
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clarityid   7671 non-null   object
 1   permid      7671 non-null   object
 2   brs_id      5453 non-null   object
 3   ovr_target  7671 non-null   object
 4   ovr_value   7652 non-null   object
dtypes: object(5)
memory usage: 299.8+ KB


In [21]:
brs_carteras

,issuer_name,aladdin_id,security_description,portfolio_full_name,portfolio_id,str_001_s,str_002_ec,str_003b_ec,str_003_ec,str_004_asec,...,str_007_sect,str_008_sec,str_009_tec,gp_esccp_22,gp_esccp_25,gp_esccp_30,gp_essccp,scs_001_sec,scs_002_ec,scs_003_sec
0,SNT-WORLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11 BIT STUDIOS SA,F79892,11 BIT STUDIOS SA,Santander Prestiz Technologii i Innowacji (San...,PLSFIO0319,Ok,Ok,Ok,Ok,Ok,...,OK,NaN,NaN,OK,Ok,Ok,OK,OK,OK,OK
2,2I RETE GAS SPA,G70839,2I RETE GAS SPA,02.018.59973438020.0,PFC00659,Ok,Ok,Ok,Ok,Excluded,...,Excluded,NaN,NaN,OK,Ok,Ok,OK,OK,OK,OK
3,2I RETE GAS SPA,G70839,2I RETE GAS SPA,ALDROVI S.L.,CPE05455,Ok,Ok,Ok,Ok,Excluded,...,Excluded,NaN,NaN,OK,Ok,Ok,OK,OK,OK,OK
4,2I RETE GAS SPA,G70839,2I RETE GAS SPA,02.018.62238191020.0,PFC00665,Ok,Ok,Ok,Ok,Excluded,...,Excluded,NaN,NaN,OK,Ok,Ok,OK,OK,OK,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76087,ZURICH INSURANCE GROUP AG,B00344,ZURICH INSURANCE GROUP AG,"SANTANDER SOSTENIBLE ACCIONES, FI",FIG05273,Ok,Ok,Ok,Ok,Ok,...,OK,NaN,NaN,OK,Ok,Ok,OK,OK,OK,OK
76088,ZURICH INSURANCE GROUP AG,B00344,ZURICH INSURANCE GROUP AG,"MI PROYECTO SANTANDER ASG 2025 PENSIONES, F.P",FPG00028,Ok,Ok,Ok,Ok,Ok,...,OK,NaN,NaN,OK,Ok,Ok,OK,OK,OK,OK
76089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76090,Confidential - For Internal Use Only. Generate...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
delta.head()

,permid,isin,issuer_name,str_001_s,str_002_ec,str_003_ec,str_004_asec,str_005_ec,cs_001_sec,gp_esccp,...,gp_esccp_22,gp_esccp_25,gp_esccp_30,art_8_basicos,str_003b_ec,new_exclusion,exclusion_list,new_inclusion,inclusion_list,aladdin_id
0,4295863735,CND100054N18,Xiangcai Co Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,[],True,[cs_003_sec],F14359
1,5000551275,CND10005MTB1,Ningbo Construction Co Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,[cs_003_sec],False,[],F69957
2,4297957993,CA48213Y1079,Jushi Holdings Inc,OK,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,[],True,"[str_001_s, str_004_asec, cs_001_sec, cs_003_s...",F73370
3,4297861403,KR6067571B83,NVH Korea Inc,OK,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,[],True,"[str_001_s, str_004_asec, cs_001_sec, cs_003_s...",G64751
4,5000063368,KR6013871CB3,GMB Korea Corp,OK,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,[],True,"[str_001_s, str_004_asec, cs_001_sec, cs_003_s...",R22919


In [25]:
# add aladdin_id to delta 
delta = delta.merge(crosreference[["permid", "aladdin_id"]], on="permid", how="left")

In [27]:
# check delta and brs_carteras columns in common
common_cols = delta.columns.intersection(brs_carteras.columns)
common_cols

Index(['issuer_name', 'str_001_s', 'str_002_ec', 'str_003_ec', 'str_004_asec',
       'str_005_ec', 'str_006_sec', 'str_007_sect', 'gp_esccp_22',
       'gp_esccp_25', 'gp_esccp_30', 'str_003b_ec', 'aladdin_id'],
      dtype='object')

In [28]:
merge_col = ['str_001_s', 'str_002_ec', 'str_003_ec', 'str_004_asec',
       'str_005_ec', 'str_006_sec', 'str_007_sect', 'gp_esccp_22',
       'gp_esccp_25', 'gp_esccp_30', 'str_003b_ec', 'aladdin_id']
# add str columns from brs
delta = delta.merge(brs_carteras[merge_col], on="aladdin_id", how="left", suffixes=("", "_brs"))

In [29]:
delta.head()

,permid,isin,issuer_name,str_001_s,str_002_ec,str_003_ec,str_004_asec,str_005_ec,cs_001_sec,gp_esccp,...,str_002_ec_brs,str_003_ec_brs,str_004_asec_brs,str_005_ec_brs,str_006_sec_brs,str_007_sect_brs,gp_esccp_22_brs,gp_esccp_25_brs,gp_esccp_30_brs,str_003b_ec_brs
0,4295863735,CND100054N18,Xiangcai Co Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5000551275,CND10005MTB1,Ningbo Construction Co Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4297957993,CA48213Y1079,Jushi Holdings Inc,OK,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4297861403,KR6067571B83,NVH Korea Inc,OK,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5000063368,KR6013871CB3,GMB Korea Corp,OK,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# SAVE RESULTS
output_file = "delta_results_beta.csv"
delta.to_csv(output_file, index=False)
logging.info(f"Results saved to {output_file}")
logging.info("Analysis completed successfully.")
